### Setting Up 

In [3]:
import pandas as pd
from datetime import datetime

### Checking File Encoding

In [4]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        rawdata = file.read()
        result = chardet.detect(rawdata)
        return result['encoding']

file_path = 'pizza_data/pizzas.csv'
encoding = detect_encoding(file_path)
print(f'The encoding of the file is: {encoding}')


The encoding of the file is: ascii


In [5]:
df_orders = pd.read_csv("pizza_data/orders.csv", encoding = "ascii")
df_pizza_types = pd.read_csv("pizza_data/pizza_types.csv", encoding = "Windows-1252")
df_order_details = pd.read_csv("pizza_data/order_details.csv", encoding = "ascii")
df_pizzas = pd.read_csv("pizza_data/pizzas.csv", encoding = "ascii")

### Data Exploration

In [24]:
# fact table
len(df_orders)

21350

In [16]:
len(df_pizza_types)

32

In [18]:
len(df_order_details)

48620

In [19]:
len(df_pizzas)

96

In [6]:
df_pizzas.head()

,pizza_id,pizza_type_id,size,price
0,bbq_ckn_s,bbq_ckn,S,12.75
1,bbq_ckn_m,bbq_ckn,M,16.75
2,bbq_ckn_l,bbq_ckn,L,20.75
3,cali_ckn_s,cali_ckn,S,12.75
4,cali_ckn_m,cali_ckn,M,16.75


In [7]:
df_order_details.head()

,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1
2,3,2,five_cheese_l,1
3,4,2,ital_supr_l,1
4,5,2,mexicana_m,1


In [8]:
df_orders.head()

,order_id,date,time
0,1,01/01/2015,11:38:36
1,2,01/01/2015,11:57:40
2,3,01/01/2015,12:12:28
3,4,01/01/2015,12:16:31
4,5,01/01/2015,12:21:30


In [9]:
df_pizza_types.head()

,pizza_type_id,name,category,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
2,ckn_alfredo,The Chicken Alfredo Pizza,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,ckn_pesto,The Chicken Pesto Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
4,southw_ckn,The Southwest Chicken Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."


In [26]:
df_orders.dtypes

order_id     int64
date        object
time        object
dtype: object

In [27]:
df_order_details.dtypes

order_details_id     int64
order_id             int64
pizza_id            object
quantity             int64
dtype: object

In [29]:
df_pizzas.dtypes

pizza_id          object
pizza_type_id     object
size              object
price            float64
dtype: object

In [30]:
df_pizza_types.dtypes

pizza_type_id    object
name             object
category         object
ingredients      object
dtype: object

In [36]:
if df_orders.isnull().values.any():
    print("There are missing values.")
else:
    print("No missing values found.")

No missing values found.


In [40]:
if df_orders.duplicated().any():
    print("There are duplicates.")
else:
    print("No duplicates found.")

No duplicates found.


In [ ]:
df['quantity'].unique()

### Data Cleaning

In [ ]:
# df_orders['datetime'] = pd.to_datetime(df_orders['date'] + ' ' + df_orders['time'])

In [10]:
# Standardize date format
df_orders['date'] = pd.to_datetime(df_orders['date'], format='%d/%m/%Y')

In [11]:
df_orders['time'] = pd.to_datetime(df_orders['time'], format='%H:%M:%S').dt.time

In [12]:
df_orders.dtypes

order_id             int64
date        datetime64[ns]
time                object
dtype: object

In [13]:
df_orders.head()

,order_id,date,time
0,1,2015-01-01,11:38:36
1,2,2015-01-01,11:57:40
2,3,2015-01-01,12:12:28
3,4,2015-01-01,12:16:31
4,5,2015-01-01,12:21:30


In [ ]:
# Convert column to appropriate data type
df_order_details['pizza_id'] = df_order_details['pizza_id'].astype(int)

### Transformations 1

df_orders 
df_pizza_types 
df_order_details 
df_pizzas 

In [59]:
df_price = pd.merge(df_order_details, df_pizzas, on="pizza_id")

In [60]:
df_price.head()

,order_details_id,order_id,pizza_id,quantity,pizza_type_id,size,price
0,1,1,hawaiian_m,1,hawaiian,M,13.25
1,2,2,classic_dlx_m,1,classic_dlx,M,16.00
2,3,2,five_cheese_l,1,five_cheese,L,18.50
3,4,2,ital_supr_l,1,ital_supr,L,20.75
4,5,2,mexicana_m,1,mexicana,M,16.00


In [63]:
df_total = df_price.groupby("order_id")['price'].sum().reset_index()

In [64]:
df_total.head()

,order_id,price
0,1,13.25
1,2,92.00
2,3,37.25
3,4,16.50
4,5,16.50


In [65]:
orders_fact_df = pd.merge(df_orders, df_total, on="order_id")

In [68]:
orders_fact_df.head(20)

,order_id,date,time,price
0,1,2015-01-01,11:38:36,13.25
1,2,2015-01-01,11:57:40,92.00
2,3,2015-01-01,12:12:28,37.25
3,4,2015-01-01,12:16:31,16.50
4,5,2015-01-01,12:21:30,16.50
5,6,2015-01-01,12:29:36,24.75
6,7,2015-01-01,12:50:37,12.50
7,8,2015-01-01,12:51:37,12.50
8,9,2015-01-01,12:52:01,143.25
9,10,2015-01-01,13:00:15,41.00


In [67]:
orders_fact_df.dtypes

order_id             int64
date        datetime64[ns]
time                object
price              float64
dtype: object

orders_fact_df -> fact
df_pizza_types -> dim
df_order_details -> dim
df_pizzas -> dim

### Transformations 2

df_orders 
df_pizza_types 
df_order_details 
df_pizzas 

In [15]:
df_fact = pd.merge(df_order_details, df_orders, on="order_id")

In [18]:
df_prices = df_pizzas[['pizza_id', 'price']]

In [36]:
df_prices.head()

,pizza_id,price
0,bbq_ckn_s,12.75
1,bbq_ckn_m,16.75
2,bbq_ckn_l,20.75
3,cali_ckn_s,12.75
4,cali_ckn_m,16.75


In [23]:
df_fact = pd.merge(df_fact, df_prices, on="pizza_id")

#### cost is price multiplied by quantity

In [25]:
df_fact['cost'] = df_fact['price'] * df_fact['quantity']

In [27]:
df_fact = df_fact.drop(columns=['price'])

In [31]:
df_fact.head()

,order_details_id,order_id,pizza_id,quantity,date,time,cost
0,1,1,hawaiian_m,1,2015-01-01,11:38:36,13.25
1,2,2,classic_dlx_m,1,2015-01-01,11:57:40,16.00
2,3,2,five_cheese_l,1,2015-01-01,11:57:40,18.50
3,4,2,ital_supr_l,1,2015-01-01,11:57:40,20.75
4,5,2,mexicana_m,1,2015-01-01,11:57:40,16.00


### Load Back

In [ ]:
https://pizza-data-project.s3.eu-west-1.amazonaws.com/order_details.csv
s3://pizza-data-project/order_details.csv